In [1]:
# ライブラリ読み込み
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import os
import pickle
from tqdm.notebook import tqdm
from scipy import stats

In [2]:
path_1 = "./folder_05/"
file_mesh = 'mesh_list_2021.pickle'
file_hpo = "hpo.csv"
file_hsdn = "hsdn.csv"

In [3]:
#hsdnはメッシュ名のネットワークなので, HPOネットワークとの比較のためMeSH_IDのネットワークに整える

#2021_mesh
with open(file_mesh, 'rb') as f:
    mesh_list = pickle.load(f)

#Zhouらの定義した疾患MeSH
all_disease = [descriptor for descriptor in mesh_list for uid in descriptor[-1] if ((uid.startswith("C")==True) & (uid.startswith("C22")!=True)&(uid.startswith("C23.888")!=True)) | (uid.startswith("F03")==True) ]
#Zhouらの定義した疾患MeSH
#all_symptom = [descriptor for descriptor in mesh_list  for uid in descriptor[-1] if (uid.startswith("C23.888")==True)]

#重複削除
disease_mesh = []
[dm for dm in all_disease if dm not in disease_mesh and not disease_mesh.append(dm)]
#symptom_mesh = []
#[ms for ms in all_symptom if ms not in symptom_mesh and not symptom_mesh.append(ms)]

print("all_mesh:", len(mesh_list), "disease_mesh:", len(disease_mesh))#, "symptom_mesh:", len(symptom_mesh))

all_mesh: 29917 disease_mesh: 4887


In [4]:
#２つのネットワークデータ 読込
hpo = pd.read_csv(path_1+file_hpo)
hsdn_df = pd.read_csv(path_1+file_hsdn)

In [5]:
#id,ターム名 取り出し
mesh_disease_id = [[i[0], i[1]] for i in disease_mesh]

#MeSH IDバージョンのHSDN作成
mesh_id_1 = pd.merge( pd.DataFrame(mesh_disease_id, columns=["mesh_id", "disease_1"]) , hsdn_df["disease_1"], how="right", on = "disease_1")
mesh_id_2 = pd.merge( pd.DataFrame(mesh_disease_id, columns=["mesh_id", "disease_2"]) , hsdn_df["disease_2"], how="right", on = "disease_2")


In [6]:
# "MeSH_ID(disease_1), MeSH_ID(disease_2), コサイン類似度" のデータフレームにする
hsdn_id = pd.concat([mesh_id_1["mesh_id"], mesh_id_2["mesh_id"]], axis=1)
hsdn = pd.concat([hsdn_id, hsdn_df["cosine_similarity"]], axis=1)
hsdn.set_axis(['disease_1', 'disease_2','cosine_similarity'], axis='columns', inplace=True)
print(len(hsdn))

7804331


In [7]:
#エッジリスト読込・ネットワーク作成

#hsdn
hsdn_G = nx.from_pandas_edgelist(hsdn, "disease_1", "disease_2" )
#benchmark
hpo_G = nx.from_pandas_edgelist( hpo,"disease_1", "disease_2" )

print("hsdn_G:", nx.info(hsdn_G), "\n")
print("hpo_G:", nx.info(hpo_G), "\n")

hsdn_G: Name: 
Type: Graph
Number of nodes: 4716
Number of edges: 7804331
Average degree: 3309.7248 

hpo_G: Name: 
Type: Graph
Number of nodes: 310
Number of edges: 9763
Average degree:  62.9871 



In [8]:
#共通ノード数: N
co_node = list(set([i for i in hsdn_G.nodes()]) & set([i for i in hpo_G.nodes()]))
N = len(co_node)
print("N:", N) 

N: 303


In [9]:
hsdn_co_node = sorted([i for i in hsdn_G.degree()  if i[0] in co_node])
hpo_co_node = sorted([i for i in hpo_G.degree()  if i[0] in co_node])
print(len(hsdn_co_node), len(hpo_co_node)) 

303 303


In [10]:
#あらかじめ、L_possible=N*(N-1)/2 を計算しておく
L_possible = N*(N - 1)/2
print("L_possible:", L_possible)

L_possible: 45753.0


In [11]:
print(len(hsdn),len(hpo))
print(hsdn[:3], "\n", hpo[:3])

7804331 9763
  disease_1 disease_2  cosine_similarity
0   D058531   D058489           0.150058
1   D000182   D058489           0.299644
2   D049913   D058165           0.058832 
   disease_1 disease_2  cosine_similarity
0   D000326   D000130           0.045566
1   D000755   D000742           0.726524
2   D000795   D000130           0.031456


In [12]:
#両者の共通部分のリンク数: L_both を求める

#共通ノードを同士のペアからなるネットワークに縮小
#Number of edges: 7804331
hsdn_index = [ enu for enu, i in tqdm(enumerate(hsdn.values.tolist())) if (i[0] in co_node) & (i[1] in co_node) ]
#Number of edges: 9763
hpo_index = [ enu for enu, i in tqdm(enumerate(hpo.values.tolist())) if (i[0] in co_node) & (i[1] in co_node) ]

hsdn_co = hsdn.iloc[hsdn_index].reset_index(drop=True)
hpo_co = hpo.iloc[hpo_index].reset_index(drop=True)

#HSDNでのリンク数を L_HSDN とする
L_HSDN = len(hsdn_co)
#HPOのリンク数を L_HPO とする
L_HPO = len(hpo_co)

print("L_HSDN:", L_HSDN)
print("L_HPO:", L_HPO)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

L_HSDN: 33371
L_HPO: 9480


In [13]:
#合わせる
same_dd_pair_1 = pd.merge(hsdn_co, hpo_co, on=["disease_1", "disease_2"])
same_dd_pair_1 = same_dd_pair_1.rename(columns={"cosine_similarity_x": "cosine_similarity_HSDN", "cosine_similarity_y": "cosine_similarity_hpo"})
print(len(same_dd_pair_1))

4842


In [14]:
#順番を逆にして合わせる
hpo_co_rev = hpo_co[["disease_2", "disease_1", "cosine_similarity"]]
hpo_co_rev = hpo_co_rev.rename(columns={"disease_2":"disease_1", "disease_1":"disease_2"})

same_dd_pair_2 = pd.merge(hsdn_co, hpo_co_rev, on=["disease_1", "disease_2"])
same_dd_pair_2 = same_dd_pair_2.rename(columns={"cosine_similarity_x": "cosine_similarity_HSDN", "cosine_similarity_y": "cosine_similarity_hpo"})
print(len(same_dd_pair_2)) 

3089


In [15]:
same_dd_pair = pd.concat([same_dd_pair_1, same_dd_pair_2], axis=0)
same_dd_pair = same_dd_pair.reset_index(drop=True)
print(len(same_dd_pair)) #6671

L_both = len(same_dd_pair)
print("L_both:", L_both)

7931
L_both: 7931


In [16]:
same_dd_pair[:3]

,disease_1,disease_2,cosine_similarity_HSDN,cosine_similarity_hpo
0,D055673,D000130,0.093254,0.131298
1,D000326,D000130,0.082216,0.045566
2,D058540,D000130,0.138097,0.312423


In [17]:
#重複チェック
same_dd_pair[same_dd_pair.duplicated()]

,disease_1,disease_2,cosine_similarity_HSDN,cosine_similarity_hpo


In [18]:
#p_HSDN=L_HSDN/L_possible を計算する
p_HSDN=L_HSDN/L_possible
#p_HPO=L_HPO/L_possible を計算する
p_HPO=L_HPO/L_possible 

print("p_HSDN:", p_HSDN)
print("p_HPO:", p_HPO)


p_HSDN: 0.7293729372937293
p_HPO: 0.20719952789980986


In [19]:
#p_HSDN * p_HPO * L_possible = L_expected を計算する
L_expected = p_HSDN * p_HPO * L_possible
print("L_expected:", L_expected)


L_expected: 6914.455445544554


In [20]:
#二項分布Binom(n, p)を仮定して、 L_both が得られる確率 P-value を求める
#二項分布のn: L_possible
#二項分布のp: L_expected/L_possible 

print("N:", N)
print("L_possible:", L_possible)
print("L_HSDN:", L_HSDN)
print("L_HPO:", L_HPO)

print("p_HSDN:", p_HSDN)
print("p_HPO:", p_HPO)
print("L_both:", L_both)
print("L_expected:", L_expected)

stats.binom_test(L_both, 
                 n=L_possible, 
                 p=L_expected/L_possible, 
                 alternative='greater')
#scipy.stats.binom_test(x, n=None, p=0.5, alternative='two-sided')[source]
#x = The number of successes
#n = The number of trials.
#p = The hypothesized probability of success 
#alternative = The default value is ‘two-sided’.


N: 303
L_possible: 45753.0
L_HSDN: 33371
L_HPO: 9480
p_HSDN: 0.7293729372937293
p_HPO: 0.20719952789980986
L_both: 7931
L_expected: 6914.455445544554


5.1630076789569366e-39

In [21]:
#参考 Zhou
Zhou_N = 898
Zhou_L_possible = 402753
Zhou_L_HSDN = 372509
Zhou_L_HPO = 111923
Zhou_p_HSDN = Zhou_L_HSDN/Zhou_L_possible
Zhou_p_HPO = Zhou_L_HPO/Zhou_L_possible
Zhou_L_both = 107098
Zhou_L_expected = Zhou_p_HSDN * Zhou_p_HPO * Zhou_L_possible

print("Zhou_N:", Zhou_N)
print("Zhou_L_possible:", Zhou_L_possible)
print("Zhou_L_HSDN:", Zhou_L_HSDN)
print("Zhou_L_HPO:", Zhou_L_HPO)
print("Zhou_p_HSDN:", Zhou_p_HSDN)
print("Zhou_p_HPO:", Zhou_p_HPO)
print("Zhou_L_both:", Zhou_L_both)
print("Zhou_L_expected:", Zhou_L_expected)

stats.binom_test(Zhou_L_both, 
                 n=Zhou_L_possible, 
                 p=Zhou_L_expected/Zhou_L_possible, 
                 alternative='greater')
#Zhou et al., (2014)ではP-value = 2.2 × 10-16 

Zhou_N: 898
Zhou_L_possible: 402753
Zhou_L_HSDN: 372509
Zhou_L_HPO: 111923
Zhou_p_HSDN: 0.9249068287511204
Zhou_p_HPO: 0.2778948884303779
Zhou_L_both: 107098
Zhou_L_expected: 103518.34699431164


3.8610006936573684e-38